Chapter 2.a.iii.  ExprRange reductions
=======

As we saw in the previous chapter, an **ExprRange** adds versatility to **Prove-It** by allowing us to define **Lambda** expressions with an indeterminant number of parameters.  Generally, **ExprRange** expressions allow us to form expressions over an indeterminant number of elements.  Basically, an **ExprRange** is how we formalize the concept of using an ellipsis in a mathematical expression.  For example, $a \cdot (b_1 + \ldots + b_n) = a \cdot b_1 + \ldots + a \cdot b_n$ (distribution) may be expressed using an **ExprRange** on each side of the equation.  Internally, the **ExprRange** has a *parameter* variable for parameterizing each element of the range.  For the LaTeX display, this *parameter* is hidden by default, as in the above example, and the precise parameterization is implicit.  To avoid ambiguity, it can be important at times to display the explicit parameterization.  For the above example, this would be $a \cdot (b_1 + ..b_k.. + b_n) = a \cdot b_1 + ..a \cdot b_k.. + a \cdot b_n$, assuming $k$ is the *parameter* in both **ExprRange** instances.

There are two kinds of reduction rules that are specific to **ExprRange** expressions that are applied in the process of performing a lambda application (beta reduction).
1. *Indexed variable expansions* occur when the **ExprRange** contains one or more **IndexedVar** expressions with an index that is parameterized over the range, and the range of indexed variables is being replaced by an **ExprTuple**.  This falls under two cases.
  1. *Parameter dependent expansion* is required whenever the parameter of the **ExprRange** occurs anywhere besides as an index of an indexed variable.  In that case, the indices of the expanded indexed variable must match the original indices.  For example, $1 \cdot x_1 + \ldots + n \cdot x_n$ could be expanded, under $(x_1, \ldots, x_n) : (a_1, \ldots, a_j, a_{j+1}, \ldots, a_n)$ to $1 \cdot a_1 + \ldots + j \cdot a_j + (j+1) \cdot a_{j+1} + \ldots + n \cdot a_n$ assuming $0 \leq j \leq n$.  However, an error would occur given $(x_1, \ldots, x_n) : (a_1, \ldots, a_j, b_1, \ldots, b_k)$ as the expansion.
  2. *Parameter independent expansion* is allowed otherwise.  In that case, only the lengths of the expansion must match, not the indices themselves.  For example, $x_1 \cdot y_1 + \ldots + x_n \cdot y_n$ could be expanded, under $(x_1, \ldots, x_n) : (a_1, \ldots, a_j, b_1, \ldots, b_k)$, $(y_1, \ldots, y_n) : (c_1, \ldots, c_j, d_1, \ldots, d_k)$ to $a_1 \cdot c_1 + \ldots + a_j \cdot c_j + b_1 \cdot d_1 + \ldots + b_k \cdot d_k$ assuming $j + k = n$.
2. *Range reductions* occur when the **ExprRange** is known to be empty or contain only a single element. 
  1. *Empty range reduction* occurs when the **ExprRange** is known to be empty and may be replaced with zero elements in a containing **ExprTuple**.
  2. *Singular element range reduction* occurs when the **ExprRange** is known to contain only a single element and may be replaced with this one element.

In [1]:
import proveit
%begin exprrange_reductions

## Creating ExprRange expressions

An `ExprRange` may be instantiated with a `parameter`, `body`, `start index`, and `end index`:

In [2]:
from proveit import ExprTuple, ExprRange, defaults
from proveit.logic import Equals
from proveit.numbers import Add, Mult, zero, one, two
from proveit import a, b, c, d, i, j, k, m, n, x, y
a_prod_range = ExprRange(k, Mult(a, k), i, j)

In [3]:
a_prod_range.parameter

_a

In [4]:
a_prod_range.body

a * _a

In [5]:
a_prod_range.start_index

i

In [6]:
a_prod_range.end_index

j

It's sub-expressions are a **Lambda** expression created from the `parameter` and `body`, the `start_index`, and the `end_index`:

In [7]:
a_prod_range.expr_info()

,core type,sub-expressions,expression
0,ExprRange,lambda_map: 1start_index: 2end_index: 3,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQUAAAAVBAMAAAC9GeaNAAAAMFBMVEX///8AAAAAAAAAAAAAAAAA AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAv3aB7AAAAD3RSTlMAInZUiRDNmWbv3US7 qzJazP2FAAAACXBIWXMAAA7EAAAOxAGVKw4bAAAC60lEQVRIx32WPYgTQRTHX3azt8nmw3CVCAcb FFIpK5xX2BjsI+EKQThJmgMbIYWIXGO6Ky924lkE7iSYQsJ1dnsH1h6HyNmd2F1jVHJNGue93Zmd 2ZndKTYz7/9+896+mcksADQg3RqZA6MhMWdIvBUyMLACzX4k9TXZ4C+go/wcbmZgsJ62tsAaJiNN XicXtdU/AEEyqDe7nbbUvxDi9NPCIcC2GGgyGQ5V29OXC4ig7bwcHmsWq0+Ia0p9V/Q0WTZUeN9a xNBuXg6vjFaGlE32clf0NEkuri/ngFACGtoIMmJtmOwr4mU1eSMzB4RWcjaE3YeMWC/wd+3hqljv FpumeMqHKBe+NX2QDOSi54BQAurNbauRADZvR9gzXNc38JUXsVhhJSuJlFH+3KguQDKQi54DQqV+ dg5YIzkSwKA8JOSA9Z8MYYfbf7lsGlssHZOdt+COQTKQi54DQvYoOwcvUCOB1bgWELLPBq+j0+Zc sUe3zCauz7kfk905eG0uo4FcwJ1OP76bTk9FDggloGHzzUQk6OFbOPCgSwjm8B6cJSoTfKC9MpJy KIfQmwl5P3bR64BQZZSfA4/kDci0E2FY7L9QSvJ/7rDJx1Lpez7sgbwW5GJYi7EMGtdCjRTVhCEn LId/UOvf5ediWWT7JKRSscbkXgDfbXEbncQuhj0ZyqDasHBVX43EDtwywjo07/3BgP+Fz6/Hx4jW rIOsPXEF14ldlByKf+KzJ0DaPDSDGOG5lSNBVBOGXOL//a2fZxdcOGfnxcP199hFhLJz9mjtjuAu Yxc5h9qPq/MISsAJX3V6THHzjVORoIZVY4h3YVi6G/S8hx5pWTbYfhpKwNR/JC7GJ22X+hFSGmTe LlhXTZYNlWEaSkC1FUG/0GrBXjdGjg3fOyHtTnpLTT7O+kgKVVBpm/ioqp9Zv4MDHmtLny5ytqiv yVsZOSAkg0qjrJxQvT9aMx6roGfdlF8vLRv8BdTM/5ZbNWP/AZO/u3gtqwBDAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
1,Lambda,parameter: 9body: 5,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAHkAAAAVCAMAAABVErFaAAAANlBMVEX///8AAAAAAAAAAAAAAAAA AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABHL6OuAAAAEXRSTlMAMrsiq4l2 RO8QZpndVM3z6UwP0jwAAAAJcEhZcwAADsQAAA7EAZUrDhsAAAEuSURBVEjHvVbblsMgCPRuvGDX ///Zjcm2DUTSE91T3hIYBkdQhZgy/UUUMgdDMO9micGOVhzniEMZhpYwxZzGK4/pnkakM5aJqpdP i0Z+wIVCmWAuZ720sU6/ei8jH/5Ktgu5GKZDnDWnhl9WSVUNXS5QvbIJhJ+lY1ykgoWlpUu5v0rh 1CF/hi6E3TgUB7RJUt1yvub08ZOR1erfTeJ7kFCZ7cdxstKWUxvYM2v2xQvKfIIwA4PjPFlzqK0B YmV6SqIDYFf7BOHExnGQqbtlM0WA7jEXNNDKdSEsM4qLivhbNlj1eirmL+b5bzAohDMct46kVE9r tQTjrNTGyR7W4r9yF4SHXKXOfubcznL8Xs8zxHeP/X+6bPatG31caDX5Mgjq2yW/ax971mzX7S9a IAoW2RVJdwAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
2,Variable,,
3,Variable,,
4,ExprTuple,9,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACUAAAAVBAMAAAAzyjqdAAAAMFBMVEX///8AAAAAAAAAAAAAAAAA AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAv3aB7AAAAD3RSTlMAInZUiRDNmWbv3US7 qzJazP2FAAAACXBIWXMAAA7EAAAOxAGVKw4bAAAAyUlEQVQY013QrwvCQBTA8e885344ZFkQtn9A DGL2H1D8AwQtgtFgMK4ZXRQsA4NVbLZpNoiI2Wyy2D0N2+4evHe8D7zH3QE+xfh3pZZih19pK0Qp AnOsGhOwIs2W4GiEM6CjWyVinnfGPQygfGWa29H3PmCP2WZkrrESEDGbzKw3bhdq74I5KaM9VOPC 7ChgJQ+RcM6txUP42Cm9zLxA7Cy5tstLbr3IOGHe+o0muHvcp/aOurziTLOFzJNKRirLUDXv9/lG oFgIXx7XJBUkcPjNAAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
5,Operation,operator: 6operands: 7,
6,Literal,,
7,ExprTuple,"8, 9","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAADkAAAAVBAMAAAAOWFv7AAAAMFBMVEX///8AAAAAAAAAAAAAAAAA AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAv3aB7AAAAD3RSTlMAInZUiRDNmWbv3US7 qzJazP2FAAAACXBIWXMAAA7EAAAOxAGVKw4bAAAA9klEQVQoz2NgYBBgwA7A4kwGOGQ3gQgTHJIM TA0MDKwJuGQZUhkY2BtwyrYzMHDilGTgDGAwwy3L1sBQAqQYrygqoIhDBFguMGQAOTsFeL6iyEIE OBIYFgOdPYOBfQGyJFSAeQLDbKCzPzBwOSDLQgX4PoBkOQ8wxG9AcSxEgHsCyOR4BYZ+FGuhAswL GA4BOQYM15gFWJFkwQIMHAcYfBkYeBSYl7MzxCOCFCLAwO7A8AroxEt+snoMXEuB7jgDBAehAgxc Gxi4HsB0mKKHlRTQywUwjgO6bAUQH4QFgQKaJOMBIBEDi2x0rTygpMOogCOKFBkYADa9MjQd2vTT AAAAAElFTkSuQmCC "" style=""display:inline;vertical-align:middle;"" />"
8,Variable,,
9,Variable,,


This `lambda_map` may be accessed directly:

In [8]:
a_prod_range.lambda_map

_a -> (a * _a)

You can create the same **ExprRange** by proving `None` for the `parameter` and `body` but suppling the `lambda_map` instead:

In [9]:
from proveit import Lambda
a_prod_range_again = ExprRange(None, None, i, j, lambda_map=Lambda(k, Mult(a, k)))

In [10]:
a_prod_range_again == a_prod_range

True

When **ExprRange**(s) are used as *operands* of an **Operation**, it is typically formatted in a clean manner that reflects that there are an indeterminant number of *operands*:

In [11]:
add_with_ranges = Add(a_prod_range, b, a_prod_range)

Note that the *operands* are an **ExprTuple** that contains the **ExprRange**(s):

In [12]:
add_with_ranges.expr_info()

,core type,sub-expressions,expression
0,Operation,operator: 1operands: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAApUAAAAVBAMAAADlUAhlAAAAMFBMVEX///8AAAAAAAAAAAAAAAAA AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAv3aB7AAAAD3RSTlMAInZUiRDNmWbv3US7 qzJazP2FAAAACXBIWXMAAA7EAAAOxAGVKw4bAAAD70lEQVRYw81YTUgbQRR+Sdz8R4OnIgqxLeil 7VZKQS8NPRZsFw+FQktyKXiR7kFK8GJO2oOgLQj9gwoWQiOV4ElPRqGHnipW2l4Klt68JLToxUtn dmc2Oz+bHUXcDrwl+Wa+977d92Z2dgAgC3wTEQj5Ai3Yo8tzvEJ4BV7oPxAd1gV8TRx6yRewWypn m2qTuVk7FU8S9dxF3+DRAQiXeSySlwBhkwMPDIBBsE2tCX6h85MQXvu85MtDWgYDF60V+Y73AI95 7L4MSLI+te9PkKwoMbUm+IVwUQwfNX15SIsQ9dxFx8qS0bO0+GlniR/hBtbpj2tITty0je3hJ5Wn XyY8pXfpKjwaNTjRCRmeIDmK0EVkgR/hBobcsmCRGNPDzxJPv0x4St80lHiLQYu+KcOjZZYe4RcC BmBljRNTkSX4ZcNT+h013njQoifxO36v33mLaQPoRjp2WHoMLzJ9t7sdIgbGSlJZo8RUZAl+YewK tMJT+sTFrz48S8to0KJxNjay6UOKd6RQCceLLB0XSuo5fDHclZPfkMqaIaYiS/ALZqLcCk/o2pFR yLbnWVpmgha9jLS+hJiz6/gdQzcSWWDpSbT4PyjDVOu1qUOn/lQqa5iYiizBbziL3zM0PKFHDiGR a8uztQwHLfoNKtqmvVPQjtDFwLI7m7iWq9UPr6tVPN8SNYBpa7cEUCjagAYv8N+pavVVtfqxJatB jOlxt3Z+NbhlkPAtOspuV96LZ4m2tTSCFo2eZaIOhRruqeALxlNcXWL6W9COrbSYBIgcSlO8QUwl xYJfsNKdYusSJXpT9+RVSOk6UYMTjeZ4IQfzLcqEhmhLQllrfyDedE8XiO4YMlkjxBSnC+vXTjcN P+RsL6eN9jxLy0jQorfRs9ThW4QeKGjHHWgdrbP0dA60v5ApXqdkBKBazstkzRFTkSX4hRBONw1P n+U+WtHb8ywtc0GLHsWdkUrMWcabF8hr30XHm5RtGDGdbzkMNELcNhDu4YRMEnP1WKsKvliLWzu/ drpj7P1mymnTh2dpmTxT0ZbWk4k+QIN37/ZddTp+oGDJGkvvRJPu0eVfu/vOE0dA5hmXyJ/vJhB3 BWxzL3gmvVR8/EIGlyQNT+9qrwQ+PEvLypmKtrSeTHRyX1LOPfxn0yo/YlX2CWHV+5Jtap9jgl9r 89PjRffmQStqcKLjpoRW4j/nZ72OH4TDAHxW4pyX+B4TcH4z+rzhCr+uynMiByx6S5LhOo+ks34A bb3E1BrnpqEvS8MDKITvDV70QxWlWt0PoG1Lnh6PxrmJDdTa3XL78FvBiw6Jh/P9Ir/bF6DfU7ap Npmb/lPxJFHPWfQ/N9CjRgx2a6IAAAAASUVORK5CYII= "" style=""display:inline;vertical-align:middle;"" />"
1,Literal,,
2,ExprTuple,"4, 3, 4","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAjwAAAAVBAMAAACwHSl+AAAAMFBMVEX///8AAAAAAAAAAAAAAAAA AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAv3aB7AAAAD3RSTlMAInZUiRDNmWbv3US7 qzJazP2FAAAACXBIWXMAAA7EAAAOxAGVKw4bAAAFEUlEQVRYw51YTWhcVRQ+8/MyM5lpGLISsTix YrtRo9RAsnFw4yo6ZFEQUiabYjbFQYKEunBWqYtAoxCoreBAK8EpluDK2SVuXBqiaBcKFXdBSDSm m4L03nPfve/ce857mc6B+7jvnO+797vn3Z/3HgDUwRTf6qk3ogMtlx7KYubSG0v35IZrBd05GEkT 0vLTpgT2HamzsIDXVm0ERG4SE2nMzmXjzp2OdyS5g1M1Ie1iXDw7D/lucsfCFxHi20EL4ILurJsl hTUFf7eQZmzxlq0VmuEgOgTHw9qhILKmCzCSJlWiJVN8uwtwxd2wMDru+q7fPlBSxlQhRG68J3gp phl731YuhbjxJsWx8CUDkTUR4tNoUqWkHnZJeuDXXY2FqaNq66+q9sodj8hN7MnQjB3bymo2joWp 43sINJU7I2lSpaKqFSlecRlnYeooNIgU2PSIQqOye9Ml+8TWNjJxPEwdr0OoaRNG0rQJM+o6I4XH XFJZeCY1PcsekduM7F62lWLPNruUiWNhz+GnZ5kSn07TMqyoqy65X6bc/h6dVyMs7tnbMKwdCOHp mQdK5KabgrNvTSYL/4W9mGaeyf7L5hgpNQPgwirFsbB2LKyK6ZmnxBRN3vASTfMut4N6zU3sYlVN 1fISzSMNawdCeHrWgBLlR1L9DH5y668EN2MaWvtK9I+bux6wOaA4FtYOhPD0rFFiiiZveImmNbij rqpEN6FkJzb8VVIjLLjVHIa1AyE8PbNAidx0b+914ZpzTOFmPGtv32zB5+acmvaBE9MfUVwY1g4D 4emZpURZkz+8RNMs3FZXVUpH5lhs61G3KmrME0cWHoa1AyFQ6ve/udXv7zkph0CJ3HRvn5iXguiR drQKJzENTQ3wP7Nfbjsgdhph2hwuDGuHgcC1fv+Lfv9b8DQdQqYmf3iJpkObnsoOtLf1U+jEDxGq G4Tvh2/HED579PSubmSn50uIHuv6lpnJvZgWv29Vj934LRA7RckDmh4vrB2FE3H2DChR1uQPL9E0 sIur3YAbCeNqpDrrkdnnh+/EEJ6eOaBEeSL/C+Uj723P0OLXntKJXSw+cGyvRXAsrFcbQnh65mgH siZ/eImmOfhBXVVpT8OvBftpFj0uqh12R9dACGsHQnh61iEm8ncf21R0DGeWXnPu57djGlL+1zuK rtQaAbC93SQ4FlYOA+HpWacdyJr84SWa1u2xV2sUtkpuHzx6Jj47cTGGYe1AiJeed+vxIamJuLHg BTeOR6QpJWeuk7zF3rBnK4IewI+mot8OPOBhrktwLKwdCPHS4zStyHLk4SWaVuBAf7kpyv47Z19x 8Qeqn3FcjF9L4YMYQtPzx1dXlecexMR+K95dcDlvkabg8ot/7j90bf2uL5qGlMtvd01lohcAz3xK cSysHQih6Uk0OWIip5M6vETTPRhXPYw/FJbks3h9A4QwdRToz4JczxILfCaLTeFpg7SEgpX70r+b Xnr4vvTWbElBB8NqUiX1i828gjZBCFNHtRt+3SGxyBuUmvq5fOI+Ch2lCCkftgR3Pf0LmnySWlLQ wbCadNkFU8IntYN7F84NFt5NOQSec8QFvg0KTU0c6xMHaYSClVpdbj4tXKtnaAo6GFaTLotxEbvL m59UYXAxRcquI/L/c2JTH36c6HMUrEQ7cvNpYQnvSEEHw2rSJdcwJbApOpPCcE7+O5mvB0RhUjZS aMwms3GTp+MdKV8fSZMqTwB7CY9i2Od21QAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
3,Variable,,
4,ExprRange,lambda_map: 5start_index: 6end_index: 7,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQUAAAAVBAMAAAC9GeaNAAAAMFBMV

For convenience, there is a `var_range` method for creating an **ExprRange** over an indexed variable.

In [13]:
from proveit import var_range, IndexedVar
var_range(x, one, n)

x_{1}, x_{2}, ..., x_{n}

Because of the `relabeling` freedom of **Lambda** expressions (alpha conversion), this is treated as `__eq__` to any similar **ExprRange** using any `parameter`.

In [14]:
var_range(x, one, n) == ExprRange(k, IndexedVar(x, k), one, n)

True

In [15]:
var_range(x, one, n) == ExprRange(a, IndexedVar(x, a), one, n)

True

Also note that there are a variety of variable ranges already defined for convenience 
in <a class="ProveItLink" href="../packages/proveit/core_expr_types/_theory_nbs_/common.ipynb">proveit.core\_expr\_types.\_theory\_nbs\_/common.ipynb</a>:

In [16]:
from proveit.core_expr_types import a_i_to_j, a_1_to_n, x_1_to_n

In [17]:
ExprTuple(a_i_to_j, a_1_to_n, x_1_to_n)

(a_{i}, a_{i + 1}, ..., a_{j},a_{1}, a_{2}, ..., a_{n},x_{1}, x_{2}, ..., x_{n})

### Showing the explicit parameterization

Some **ExprRanges** can be ambiguous in the default formatting style.  For example, the following expression are different but have identical default formatting.  UPDATE: We now require indices to be in a canonical form which presents this issue.

In [18]:
range__add1__0_to_n = ExprRange(k, Add(k, one), zero, n)

In [19]:
try:
    range__0_plus_1__to__n_plus_1 = ExprRange(k, k, Add(zero, one), Add(n, one))
    assert False, "Expected ValueError"
except ValueError as e:
    print("Expected Error:", e)

Expected Error: Start and end indices must be in the 'quick_simplified_index' canonical form: (0 + 1, n + 1) ≠ (1, n + 1)


In [20]:
range__1__to__n_plus_1 = ExprRange(k, k, one, Add(n, one))

In [21]:
range__add1__0_to_n==range__1__to__n_plus_1

False

We have an alternate *style* for formatting an **ExprRange** without this ambiguity.  This style is called *explicit parameterization*.  In between the first and last of the range, it shows the body of the lambda map of the **ExprRange**.

In [22]:
range__add1__0_to_n.with_explicit_parameterization()

(0 + 1), (1 + 1), ..(_a + 1).., (n + 1)

In [23]:
range__1__to__n_plus_1.with_explicit_parameterization()

1, 2, .._a.., (n + 1)

Because it is less compact and not typically necessary, this is not the default style for LaTeX formatting.  However, it is the default for "string" formatting that gets used in error messages for the sake of clarity and making the debugging process a little easier.

### Simple repetitions

When an **ExprRange** has a lambda *body* that is independent of its parameter, it requires special formatting to avoid ambiguity.  The simple case in when the start index is $1$.

In [24]:
ExprRange(k, Add(one, two), one, n)

(1 + 2), (1 + 2), ..(n - 3) ×.., (1 + 2)

Otherwise, the start and end indices will be provided when the **ExprRange** is formatted.

In [25]:
ExprRange(k, Add(one, two), i, j)

(1 + 2), (1 + 2), ..(j - i - 2) ×.., (1 + 2)

### Nested ExprRanges

**ExprRange**s may be nested to express a range of ranges representing elements of a single **ExprTuple**.  For clarity of the expression, ellipses are doubled to express a range of ranges (or tripled for a range of ranges of ranges, etc.).

In [26]:
from proveit.numbers import one
double_nested = ExprRange(i, ExprRange(j, Mult(i, j), one, n), one, n)

In [27]:
ExprTuple(double_nested)

((1 * 1), (1 * 2), ..., (1 * n), (2 * 1), (2 * 2), ..., (2 * n), ......, (n * 1), (n * 2), ..., (n * n))

If $n = 3$, for example, the above **ExprTuple** could expand to $((1 \cdot 1), (1 \cdot 2), (1 \cdot 3), (2 \cdot 1), (2 \cdot 2), (2 \cdot 3), (3 \cdot 1), (3 \cdot 2), (3 \cdot 3))$.

Let's now show triple nesting for fun:

In [28]:
triple_nested = ExprRange(i, ExprRange(j, ExprRange(k, Mult(i, j, k), 
                                                    one, n), 
                                       one, n), 
                          one, n)

In [29]:
Add(triple_nested)

(1 * 1 * 1) +  (1 * 1 * 2) +  ... +  (1 * 1 * n) +  (1 * 2 * 1) +  (1 * 2 * 2) +  ... +  (1 * 2 * n) +  ...... +  (1 * n * 1) +  (1 * n * 2) +  ... +  (1 * n * n) +  (2 * 1 * 1) +  (2 * 1 * 2) +  ... +  (2 * 1 * n) +  (2 * 2 * 1) +  (2 * 2 * 2) +  ... +  (2 * 2 * n) +  ...... +  (2 * n * 1) +  (2 * n * 2) +  ... +  (2 * n * n) +  ......... +  (n * 1 * 1) +  (n * 1 * 2) +  ... +  (n * 1 * n) +  (n * 2 * 1) +  (n * 2 * 2) +  ... +  (n * 2 * n) +  ...... +  (n * n * 1) +  (n * n * 2) +  ... +  (n * n * n)

The `var_range` function may also be used to make nested ranges of indexed variables by simply supplying multiple start and end indices.

In [30]:
x_111_to_nnn = var_range(x, (one, one, one), (n, n, n))

Note that multiple indices are implemented by nesting **IndexedVar** expressions:

In [31]:
x_111_to_nnn.body.body.body.expr_info()

,core type,sub-expressions,expression
0,IndexedVar,variable: 1indices: 2,"<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAFIAAAAPBAMAAACFEoWBAAAAMFBMVEX///8AAAAAAAAAAAAAAAAA AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAv3aB7AAAAD3RSTlMAEHarMolUmd1mIs3v RLtmgJDEAAAACXBIWXMAAA7EAAAOxAGVKw4bAAAA2UlEQVQoz2NgEFIWCU1gIAYUMBmxNxCjkHMB 50fmCVgkvB+gCbAwsG/AagRzAIYQjwJWlTwOGEL3FzAKQJmsmghd/kqGcDZjpOoCBrkLNgysMJFe lgNwWQ0BdThbYoH/AoZ8xxqGa1AB9geMCCtfMsyGs8sZfBgYxJXYKmHByXMByVFHGIrh7LMMyqgu 5l+AxPnAcAzuzF8MLwVRVHJfYAhghAVOI+cDOPsUy9+JUCZXeXl5LQNX5SQBlmMMy4GcCwzuSgxw 9vVpqlgihw0HGxNcxMHGBBNwsAF8zzJloQZ0vQAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
1,Variable,,
2,ExprTuple,"3, 4, 5","<img src=""data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAGUAAAAVBAMAAACtcD09AAAAMFBMVEX///8AAAAAAAAAAAAAAAAA AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAv3aB7AAAAD3RSTlMAInZUiRDNmWbv3US7 qzJazP2FAAAACXBIWXMAAA7EAAAOxAGVKw4bAAABVElEQVQ4y5VTMUvDUBD+0jS+JkabuSCkguAk GdQ5iJNQyQ8QkkVwETo4uJlNNzsKLgUV1+Dm1LSb4CC2OGd26hJnLw7NvZgU8obA9919ed/dvQNg oc75y244tTQv2We3Kqq9DUvYRkiRoPKXK/0y9hQQYaWmXer6GtCrrY+8Mlb3sF+tOSp3HOKCwYPt hEfPN6dStjLr2kDzA2c513FEzPv24/nS7F4tMwVaAR5y7hG6zVLUVMbaHQR1Xx3gfsGJVDYuArRd jucwCK/PmUaPZQ0ljHi3Ke5HwOqAefPtf+O54t2m+G1meYhJnkPTjTSmSagChn0HX6qFVoxe7jdG M/Hzp7QWmn0wbNrqs6A0F99U1TudMXB4swXjaYG12SUk/Hm8sUNMBCORq9jDcowOjajweF0sx6DL MZaXptC8IoaSTeREXqniihWvMbPnpNi1drsL/AKUK0gUmc44ywAAAABJRU5ErkJggg== "" style=""display:inline;vertical-align:middle;"" />"
3,Variable,,
4,Variable,,
5,Variable,,


## Expanding ExprRanges

The **ExprRange** and **IndexedVar** expression types work together in **ExprRange** expansion reduction rule.  Essentially, when a range of indexed variables is instantiated to an **ExprTuple** of elements and those indexed variables are contained within an **ExprRange**, that **ExprRange** must be "expanded" to reflect the expansion of the indexed variables.

### Parameter dependent expansions

The more restrictive form of **ExprRange** expansion is when **ExprRange** parameter is used for anything other than indices of indexed variables being expanded.  For example, consider the following lambda expression.

In [32]:
param_dep_lambda_example = Lambda(x_1_to_n, Add(ExprRange(k, Mult(k, IndexedVar(x, k)), 
                                                          one, n)))

Note that the **ExprRange** parameter on the right appears as a prefactor as well as an index of $x$.  To respect the restrictions of a parameter dependent expansion, in this case, we may only apply this lambda map to an **ExprTuple** with **ExprRanges** whose indices match the original indices.  We demonstrate this below by applying `param_dep_lambda_example` to $a_1, \ldots, a_j, a_{j+1}, \ldots, a_n$.  First, we prove a few things in order to meet the necessary *requirements* for the expansion under a set of *assumption*

In [33]:
from proveit.core_expr_types import Len
from proveit.logic import InSet
from proveit.numbers import Natural, subtract
param_dep_assumptions = [InSet(j, Natural), InSet(subtract(n, j), Natural),
                         InSet(n, Natural)]
defaults.assumptions = param_dep_assumptions

In [34]:
a_1_to_n_partition = a_1_to_n.partition(j)

In [35]:
typical = Len(a_1_to_n_partition.lhs).typical_eq()

from proveit.logic import InSet
from proveit import j
from proveit.numbers import Natural
InSet(j, Natural).prove()

In [36]:
Len(a_1_to_n_partition.rhs).deduce_equal(typical.rhs)

{j in Natural, (n - j) in Natural, n in Natural} |- |(a_{1}, a_{2}, ..., a_{j},a_{j + 1}, a_{j + 2}, ..., a_{n})| = |(1, 2, ..., n)|

In [37]:
typical.rhs.operands.entries[0].partition(j)

{j in Natural, (n - j) in Natural} |- (1, 2, ..., n) =  \\ (1, 2, ..., j,(j + 1), (j + 2), ..., n)

In [38]:
param_dep_operands = a_1_to_n_partition.rhs

Now we will apply `param_dep_lambda_example` to $a_1, \ldots, a_j, a_{j+1}, \ldots, a_n$

In [39]:
requirements = []
param_dep_lambda_example.apply(*param_dep_operands.entries, requirements = requirements)

(1 * a_{1}) +  (2 * a_{2}) +  ... +  (j * a_{j})+ ((j + 1) * a_{j + 1}) +  ((j + 2) * a_{j + 2}) +  ... +  (n * a_{n})

The following requirements are needed for this lambda application.  First, the length of the operands must match the length of the parameter index range for a proper applications (as we saw in the previous chapter).  Second, the indices themselves must match in order to perform this parameter-dependent **ExprRange** expansion.

In [40]:
requirements

(|(a_{1}, a_{2}, ..., a_{j},a_{j + 1}, a_{j + 2}, ..., a_{n})| = |(1, 2, ..., n)|, 1 in Integer, n in Integer, (1, 2, ..., j,(j + 1), (j + 2), ..., n) = (1, 2, ..., n))

### Parameter independent expansions

When the **ExprRange** parameter only appears as indices of variables that are being expanded, the **ExprRange** expansion reduction is more permissive.  Indices do not need to match.  Consider the following `dot_prod_lambda` example.

In [41]:
from proveit.core_expr_types import (
    a_1_to_j, b_1_to_k, c_1_to_j, d_1_to_k, x_1_to_n, y_1_to_n)
dot_prod_lambda = \
    Lambda((x_1_to_n, y_1_to_n), 
           Add(ExprRange(a, Mult(IndexedVar(x, a),
                                 IndexedVar(y, a)), one, n)))

We map expand the $x_k$ variables and $y_k$ variables in any manner, with respecting the original indices, as long as the expansions of these two match.  We will demonstrate this by applying `dot_prod_lambda` such that the $x$ operands will be $(a_1, \dots, a_j, b_1, \ldots, b_k)$ and the $y$ operands will be $(c_1, \ldots, c_j, 1 \cdot d_1, \ldots, k \cdot d_k)$.  These don't respect the original indices since they go up to $j$ and then restart at $1$, but the do match each other, both going $1$ to $j$ then $1$ to $k$.  First, we will make some assumptions and prove a few things so we can meet the requirements.

In [42]:
jpk_eq_n = Equals(Add(j, k), n)

In [43]:
param_indep_assumptions = [InSet(j, Natural), InSet(k, Natural), jpk_eq_n]
defaults.assumptions = param_indep_assumptions

In [44]:
jpk_in_nats = InSet(Add(j, k), Natural).prove()

In [45]:
jpk_eq_n.sub_right_side_into(jpk_in_nats)

{j in Natural, k in Natural, (j + k) = n} |- n in Natural

In [46]:
x_operands = [a_1_to_j, b_1_to_k]

Make this one a little more interesting and demonstrate that expansions can be **ExprRange**s of any kind, not just variable ranges.

In [47]:
y_operands = [c_1_to_j, ExprRange(i, Mult(i, IndexedVar(d, i)), one, k)]

In [48]:
x_len_jpk = Len(x_operands).computation()

In [49]:
jpk_eq_n.sub_right_side_into(x_len_jpk)

{j in Natural, k in Natural, (j + k) = n} |- |(a_{1}, a_{2}, ..., a_{j},b_{1}, b_{2}, ..., b_{k})| = n

In [50]:
y_len_jpk = Len(y_operands).computation()

In [51]:
jpk_eq_n.sub_right_side_into(y_len_jpk)

{j in Natural, k in Natural, (j + k) = n} |- |(c_{1}, c_{2}, ..., c_{j},(1 * d_{1}), (2 * d_{2}), ..., (k * d_{k}))| = n

Now we perform the lambda application

In [52]:
requirements = []
dot_prod_lambda.apply(*x_operands, *y_operands, requirements=requirements)

(a_{1} * c_{1}) +  (a_{2} * c_{2}) +  ... +  (a_{j} * c_{j})+ (b_{1} * (1 * d_{1})) +  (b_{2} * (2 * d_{2})) +  ... +  (b_{k} * (k * d_{k}))

Note that the requirements, in this case, only relate to matching the length of the operands and respective parameter index ranges.  The necessity that the $x$ operands and the $y$ operands match with respect to corresponding **ExprRange** start and end indices is not needed as an explicit requirement; it is a simple matter of checking that the expressions of these indices match identically.  In other words, that requirement is met by a trivial inspection.

In [53]:
requirements

(|(a_{1}, a_{2}, ..., a_{j},b_{1}, b_{2}, ..., b_{k})| = |(1, 2, ..., n)|, |(c_{1}, c_{2}, ..., c_{j},(1 * d_{1}), (2 * d_{2}), ..., (k * d_{k}))| = |(1, 2, ..., n)|)

Parameter independent expansions are only possible when all of the indexed variables of a particular expression range are expanded.

In [54]:
dot_prod_with_y = \
    Lambda(x_1_to_n, 
           Add(ExprRange(a, Mult(IndexedVar(x, a),
                                 IndexedVar(y, a)), one, n)))

In [55]:
from proveit import LambdaApplicationError
try:
    dot_prod_with_y.apply(a_1_to_j, b_1_to_k, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, x_{2}, ..., x_{n}) -> ((x_{1} * y_{1}) +  (x_{2} * y_{2}) +  ... +  (x_{n} * y_{n})) to (a_{1}, a_{2}, ..., a_{j}, b_{1}, b_{2}, ..., b_{k}) assuming {j in Natural, k in Natural, (j + k) = n}:
Improper replacement: Improper replacement of (x_{1} * y_{1}), (x_{2} * y_{2}), ..., (x_{n} * y_{n}) via {(x_{1}, x_{2}, ..., x_{n}): (a_{1}, a_{2}, ..., a_{j},b_{1}, b_{2}, ..., b_{k}), x: {(x_{1}, x_{2}, ..., x_{n})}}:
ExprRange indices failed to match expansion which is necessary because not all of the indexed variables being indexed by the ExprRange parameter are being expanded (x is expanded but y is not): Unable to prove (1, 2, ..., j,(1 + j), (2 + j), ..., (k + j)) = (1, 2, ..., n) assuming {j in Natural, k in Natural, (j + k) = n}:
Insufficient known transitive relations to sort the remaining items: {(1, 2, ..., j,(1 + j), (2 + j), ..., (k + j)), (1, 2, ..., n)}. 


But we can do a parameter dependent expansion in this case, as long as we can prove that the indices of the expansion match the original indices.

In [56]:
requirements = []
dot_prod_with_y.apply(*param_dep_operands.entries, assumptions=param_dep_assumptions, 
                      requirements=requirements)

(a_{1} * y_{1}) +  (a_{2} * y_{2}) +  ... +  (a_{j} * y_{j})+ (a_{j + 1} * y_{j + 1}) +  (a_{j + 2} * y_{j + 2}) +  ... +  (a_{n} * y_{n})

In [57]:
requirements

(|(a_{1}, a_{2}, ..., a_{j},a_{j + 1}, a_{j + 2}, ..., a_{n})| = |(1, 2, ..., n)|, 1 in Integer, n in Integer, (1, 2, ..., j,(j + 1), (j + 2), ..., n) = (1, 2, ..., n))

Also note that an explicit expansion must be provided for each precise range containing an expanded range of variables.

In [58]:
inconsistent_dot_prod = \
    Lambda((x_1_to_n, y_1_to_n), 
           Add(ExprRange(a, Mult(IndexedVar(x, a),
                                 IndexedVar(y, a)), one, m)))

In [59]:
try:
    inconsistent_dot_prod.apply(*x_operands, *y_operands, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, x_{2}, ..., x_{n}, y_{1}, y_{2}, ..., y_{n}) -> ((x_{1} * y_{1}) +  (x_{2} * y_{2}) +  ... +  (x_{m} * y_{m})) to (a_{1}, a_{2}, ..., a_{j}, b_{1}, b_{2}, ..., b_{k}, c_{1}, c_{2}, ..., c_{j}, (1 * d_{1}), (2 * d_{2}), ..., (k * d_{k})) assuming {j in Natural, k in Natural, (j + k) = n}:
Improper replacement: Improper replacement of (x_{1} * y_{1}), (x_{2} * y_{2}), ..., (x_{m} * y_{m}) via {(x_{1}, x_{2}, ..., x_{n}): (a_{1}, a_{2}, ..., a_{j},b_{1}, b_{2}, ..., b_{k}), (y_{1}, y_{2}, ..., y_{n}): (c_{1}, c_{2}, ..., c_{j},(1 * d_{1}), (2 * d_{2}), ..., (k * d_{k})), x: {(x_{1}, x_{2}, ..., x_{n})}, y: {(y_{1}, y_{2}, ..., y_{n})}}:
Failure to expand (x_{1} * y_{1}), (x_{2} * y_{2}), ..., (x_{m} * y_{m}) because there is no explicit expansion for (x_{1}, x_{2}, ..., x_{m}).  The known expansions for this variable are {(x_{1}, x_{2}, ..., x_{n}): (a_{1}, a_{2}, ..., a_{j},b_{1}, b_{2}, ..., b_{k})}.
(Note that multiple, equivalent expansion form

However, we may supply *equivalent alternative expansions*, `equiv_alt_expansions`, to handle such cases as appropriate.  This was covered in the previous chapter and will not be revisited here.

All of the expansions within an **ExprRange** must match with respect to the indices of their **ExprRanges**.  That means that the number of entries of the expansions must match:

In [60]:
try:
    dot_prod_lambda.apply(a_1_to_j, b_1_to_k, y_1_to_n, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, x_{2}, ..., x_{n}, y_{1}, y_{2}, ..., y_{n}) -> ((x_{1} * y_{1}) +  (x_{2} * y_{2}) +  ... +  (x_{n} * y_{n})) to (a_{1}, a_{2}, ..., a_{j}, b_{1}, b_{2}, ..., b_{k}, y_{1}, y_{2}, ..., y_{n}) assuming {j in Natural, k in Natural, (j + k) = n}:
Improper replacement: Improper replacement of (x_{1} * y_{1}), (x_{2} * y_{2}), ..., (x_{n} * y_{n}) via {(x_{1}, x_{2}, ..., x_{n}): (a_{1}, a_{2}, ..., a_{j},b_{1}, b_{2}, ..., b_{k}), (y_{1}, y_{2}, ..., y_{n}): (y_{1}, y_{2}, ..., y_{n}), x: {(x_{1}, x_{2}, ..., x_{n})}, y: {(y_{1}, y_{2}, ..., y_{n})}}:
When expanding IndexedVars within an ExprRange whose parameter is the index, their expansion ExprRange indices must all match. (a_{1}, a_{2}, ..., a_{j}, b_{1}, b_{2}, ..., b_{k}) vs (y_{1}, y_{2}, ..., y_{n},) do not match as respective expansions for x_{_a} and y_{_a}.
ExprTuple.align_ranges may be useful.
Using multiple, equivalent expansion forms could also help. 


They must match with respect to which entries are singular versus **ExprRanges**:

In [61]:
from proveit.numbers import NaturalPos
defaults.assumptions = [InSet(n, NaturalPos)] + param_indep_assumptions

In [62]:
x_partition = x_1_to_n.partition(one)

In [63]:
x_operands_new = x_partition.rhs

In [64]:
dot_prod_lambda

(x_{1}, x_{2}, ..., x_{n}, y_{1}, y_{2}, ..., y_{n}) -> ((x_{1} * y_{1}) +  (x_{2} * y_{2}) +  ... +  (x_{n} * y_{n}))

In [65]:
try:
    dot_prod_lambda.apply(*x_operands_new.entries, a_1_to_j, b_1_to_k, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, x_{2}, ..., x_{n}, y_{1}, y_{2}, ..., y_{n}) -> ((x_{1} * y_{1}) +  (x_{2} * y_{2}) +  ... +  (x_{n} * y_{n})) to (x_{1}, x_{2}, x_{3}, ..., x_{n}, a_{1}, a_{2}, ..., a_{j}, b_{1}, b_{2}, ..., b_{k}) assuming {n in NaturalPos, j in Natural, k in Natural, (j + k) = n}:
Improper replacement: Improper replacement of (x_{1} * y_{1}), (x_{2} * y_{2}), ..., (x_{n} * y_{n}) via {(x_{1}, x_{2}, ..., x_{n}): (x_{1},x_{2}, x_{3}, ..., x_{n}), (y_{1}, y_{2}, ..., y_{n}): (a_{1}, a_{2}, ..., a_{j},b_{1}, b_{2}, ..., b_{k}), x: {(x_{1}, x_{2}, ..., x_{n})}, y: {(y_{1}, y_{2}, ..., y_{n})}}:
When expanding IndexedVars within an ExprRange whose parameter is the index, their expansion ExprRange indices must all match. (x_{1}, x_{2}, x_{3}, ..., x_{n}) vs (a_{1}, a_{2}, ..., a_{j}, b_{1}, b_{2}, ..., b_{k}) do not match as respective expansions for x_{_a} and y_{_a}.
ExprTuple.align_ranges may be useful.
Using multiple, equivalent expansion forms could also help

Each of the start indices must match:

In [66]:
from proveit.numbers import zero
x_shift_equiv = x_1_to_n.shift_equivalence(new_start=zero)

In [67]:
x_operands_shifted = x_shift_equiv.rhs

In [68]:
try:
    dot_prod_lambda.apply(*x_operands_shifted.entries, y_1_to_n, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, x_{2}, ..., x_{n}, y_{1}, y_{2}, ..., y_{n}) -> ((x_{1} * y_{1}) +  (x_{2} * y_{2}) +  ... +  (x_{n} * y_{n})) to (x_{0 + 1}, x_{1 + 1}, ..., x_{(n - 1) + 1}, y_{1}, y_{2}, ..., y_{n}) assuming {n in NaturalPos, j in Natural, k in Natural, (j + k) = n}:
Improper replacement: Improper replacement of (x_{1} * y_{1}), (x_{2} * y_{2}), ..., (x_{n} * y_{n}) via {(x_{1}, x_{2}, ..., x_{n}): (x_{0 + 1}, x_{1 + 1}, ..., x_{(n - 1) + 1}), (y_{1}, y_{2}, ..., y_{n}): (y_{1}, y_{2}, ..., y_{n}), x: {(x_{1}, x_{2}, ..., x_{n})}, y: {(y_{1}, y_{2}, ..., y_{n})}}:
When expanding IndexedVars within an ExprRange whose parameter is the index, their expansion ExprRange indices must all match. (x_{0 + 1}, x_{1 + 1}, ..., x_{(n - 1) + 1},) vs (y_{1}, y_{2}, ..., y_{n},) do not match as respective expansions for x_{_a} and y_{_a}.
ExprTuple.align_ranges may be useful.
Using multiple, equivalent expansion forms could also help. 


And each of the end indices must match:

In [69]:
defaults.assumptions = param_indep_assumptions

In [70]:
y_operands_new = [b_1_to_k, a_1_to_j]

In [71]:
ynew_len_kpj = Len(y_operands_new).computation()

In [72]:
ynew_len_jpk = ynew_len_kpj.inner_expr().rhs.commute()

In [73]:
jpk_eq_n.sub_right_side_into(ynew_len_jpk)

{k in Natural, j in Natural, (j + k) = n} |- |(b_{1}, b_{2}, ..., b_{k},a_{1}, a_{2}, ..., a_{j})| = n

In [74]:
try:
    dot_prod_lambda.apply(*x_operands, *y_operands_new, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1}, x_{2}, ..., x_{n}, y_{1}, y_{2}, ..., y_{n}) -> ((x_{1} * y_{1}) +  (x_{2} * y_{2}) +  ... +  (x_{n} * y_{n})) to (a_{1}, a_{2}, ..., a_{j}, b_{1}, b_{2}, ..., b_{k}, b_{1}, b_{2}, ..., b_{k}, a_{1}, a_{2}, ..., a_{j}) assuming {j in Natural, k in Natural, (j + k) = n}:
Improper replacement: Improper replacement of (x_{1} * y_{1}), (x_{2} * y_{2}), ..., (x_{n} * y_{n}) via {(x_{1}, x_{2}, ..., x_{n}): (a_{1}, a_{2}, ..., a_{j},b_{1}, b_{2}, ..., b_{k}), (y_{1}, y_{2}, ..., y_{n}): (b_{1}, b_{2}, ..., b_{k},a_{1}, a_{2}, ..., a_{j}), x: {(x_{1}, x_{2}, ..., x_{n})}, y: {(y_{1}, y_{2}, ..., y_{n})}}:
When expanding IndexedVars within an ExprRange whose parameter is the index, their expansion ExprRange indices must all match. (a_{1}, a_{2}, ..., a_{j}, b_{1}, b_{2}, ..., b_{k}) vs (b_{1}, b_{2}, ..., b_{k}, a_{1}, a_{2}, ..., a_{j}) do not match as respective expansions for x_{_a} and y_{_a}.
ExprTuple.align_ranges may be useful.
Using multiple, eq

## Nested ExprRange expansions

As we saw above, we may have nested **ExprRange** expressions and ranges of ranges of variables/parameters.

In [75]:
x_11_to_mn = var_range(x, (one, one), (m, n))

Consider the following two different lambda expressions mapping a range of ranges of parameters.  The first multiplies groups of sums of terms and the second is a range of ranges of terms added together. 

In [76]:
double_nested_lambda1 = Lambda(x_11_to_mn, Mult(
    ExprRange(i, Add(ExprRange(j, Mult(i, j, IndexedVar(x, (i, j))), 
                               one, n)), 
              one, m)))

In [77]:
double_nested_lambda2 = Lambda(x_11_to_mn, Add(
    ExprRange(i, ExprRange(j, Mult(i, j, IndexedVar(x, (i, j))), 
                           one, n), 
              one, m)))

First, let's consider a simple example in which we assume that $m=2$ and $n=2$ and apply `double_nested_lambda` to $(a, b, c, d)$.

In [78]:
abcd = ExprTuple(a, b, c, d)

We'll make some assumptions and prove a few things to meet the requirements.

In [79]:
from proveit.numbers import two
defaults.assumptions = [Equals(n, two), Equals(m, two)]

In [80]:
Mult(m, n).evaluation()

{m = 2, n = 2} |- (m * n) = 4

In [81]:
from proveit import extract_var_tuple_indices
Equals(Len(abcd), Len(extract_var_tuple_indices(ExprTuple(x_11_to_mn)))).prove()

{m = 2, n = 2} |- |(a, b, c, d)| = |((1, 1), (1, 2), ..., (1, n), (2, 1), (2, 2), ..., (2, n), ......, (m, 1), (m, 2), ..., (m, n))|

In [82]:
eq = Equals(ExprTuple(one, two), ExprTuple(ExprRange(k, k, one, two))).prove()

In [83]:
eq.inner_expr().rhs[0].end_index.substitute(n)

{n = 2} |- (1, 2) = (1, 2, ..., n)

In [84]:
Equals(n, two).sub_left_side_into(eq.inner_expr().rhs[0].end_index, 
                                  auto_simplify=False)

{n = 2} |- (1, 2) = (1, 2, ..., n)

In [85]:
Equals(m, two).sub_left_side_into(eq.inner_expr().rhs[0].end_index)

{m = 2} |- (1, 2) = (1, 2, ..., m)

Now we will apply `double_nested_lambda1` to $(a, b, c, d)$ and demonstrate the expansion of nested **ExprRanges**.

In [86]:
double_nested_lambda1

(x_{1, 1}, x_{1, 2}, ..., x_{1, n}, x_{2, 1}, x_{2, 2}, ..., x_{2, n}, ......, x_{m, 1}, x_{m, 2}, ..., x_{m, n}) -> (((1 * 1 * x_{1, 1}) +  (1 * 2 * x_{1, 2}) +  ... +  (1 * n * x_{1, n})) *  ((2 * 1 * x_{2, 1}) +  (2 * 2 * x_{2, 2}) +  ... +  (2 * n * x_{2, n})) *  ... *  ((m * 1 * x_{m, 1}) +  (m * 2 * x_{m, 2}) +  ... +  (m * n * x_{m, n})))

In [87]:
requirements=[]
double_nested_lambda1.apply(*abcd.entries, requirements=requirements)

((1 * 1 * a) + (1 * 2 * b)) * ((2 * 1 * c) + (2 * 2 * d))

Note in the requirements below that we need indices to match, (1,2)=(1,…,𝑛)
and (1,2)=(1,…,𝑚) in order to perform this parameter-dependent expansion.

In [88]:
# remove duplicate requirements using an OrderedDict
from collections import OrderedDict
requirements = list(OrderedDict.fromkeys(requirements))
requirements

(|(a, b, c, d)| = |((1, 1), (1, 2), ..., (1, n), (2, 1), (2, 2), ..., (2, n), ......, (m, 1), (m, 2), ..., (m, n))|, |(a, b)| = |((_b, 1), (_b, 2), ..., (_b, n))|, |(c, d)| = |((_b, 1), (_b, 2), ..., (_b, n))|, 1 in Integer, n in Integer, (1, 2) = (1, 2, ..., n), m in Integer, (1, 2) = (1, 2, ..., m))

Now we will apply `double_nested_lambda2` to $(a, b, c, d)$ and demonstrate the expansion of nested **ExprRanges**.

In [89]:
requirements=[]
double_nested_lambda2.apply(*abcd.entries, requirements=requirements)

(1 * 1 * a) + (1 * 2 * b) + (2 * 1 * c) + (2 * 2 * d)

Note in the requirements below that we need indices to match, $(1, 2) = (1, \ldots, n)$ and $(1, 2) = (1, \ldots, m)$ in order to perform this parameter-dependent expansion.

In [90]:
# remove duplicate requirements using an OrderedDict
from collections import OrderedDict
requirements = list(OrderedDict.fromkeys(requirements))
requirements

(|(a, b, c, d)| = |((1, 1), (1, 2), ..., (1, n), (2, 1), (2, 2), ..., (2, n), ......, (m, 1), (m, 2), ..., (m, n))|, |(a, b)| = |((_b, 1), (_b, 2), ..., (_b, n))|, |(c, d)| = |((_b, 1), (_b, 2), ..., (_b, n))|, 1 in Integer, n in Integer, (1, 2) = (1, 2, ..., n), m in Integer, (1, 2) = (1, 2, ..., m))

Let's do one more example to test this capability

In [91]:
double_nested_operands = ExprTuple(var_range(x, (one, one), (subtract(m, one), n)),
                                   ExprRange(k, Mult(k, k), one, n))

In [92]:
defaults.assumptions = [InSet(m, NaturalPos), InSet(n, Natural)]

In [93]:
ExprRange(k, k, one, m).partition(subtract(m, one))

{m in NaturalPos} |- (1, 2, ..., m) =  \\ (1, 2, ..., (m - 1), m)

In [94]:
Add(Mult(n, subtract(m, one)), n).inner_expr().operands[0].distribution(1).inner_expr().rhs.commute()

{n in Natural, m in NaturalPos} |- ((n * (m - 1)) + n) = (m * n)

In [95]:
requirements=[]
double_nested_lambda1.apply(*double_nested_operands.entries, requirements=requirements)

((1 * 1 * x_{1, 1}) +  (1 * 2 * x_{1, 2}) +  ... +  (1 * n * x_{1, n})) *  ((2 * 1 * x_{2, 1}) +  (2 * 2 * x_{2, 2}) +  ... +  (2 * n * x_{2, n})) *  ... *  (((m - 1) * 1 * x_{m - 1, 1}) +  ((m - 1) * 2 * x_{m - 1, 2}) +  ... +  ((m - 1) * n * x_{m - 1, n})) * ((m * 1 * (1 * 1)) +  (m * 2 * (2 * 2)) +  ... +  (m * n * (n * n)))

In [96]:
requirements

(|(x_{1, 1}, x_{1, 2}, ..., x_{1, n}, x_{2, 1}, x_{2, 2}, ..., x_{2, n}, ......, x_{m - 1, 1}, x_{m - 1, 2}, ..., x_{m - 1, n},(1 * 1), (2 * 2), ..., (n * n))| = |((1, 1), (1, 2), ..., (1, n), (2, 1), (2, 2), ..., (2, n), ......, (m, 1), (m, 2), ..., (m, n))|, |((1 * 1), (2 * 2), ..., (n * n))| = |((_b, 1), (_b, 2), ..., (_b, n))|, 1 in Integer, n in Integer, 1 in Integer, m in Integer, (1, 2, ..., (m - 1), m) = (1, 2, ..., m))

In [97]:
requirements=[]
double_nested_lambda2.apply(*double_nested_operands.entries, requirements=requirements)

(1 * 1 * x_{1, 1}) +  (1 * 2 * x_{1, 2}) +  ... +  (1 * n * x_{1, n}) +  (2 * 1 * x_{2, 1}) +  (2 * 2 * x_{2, 2}) +  ... +  (2 * n * x_{2, n}) +  ...... +  ((m - 1) * 1 * x_{m - 1, 1}) +  ((m - 1) * 2 * x_{m - 1, 2}) +  ... +  ((m - 1) * n * x_{m - 1, n})+ (m * 1 * (1 * 1)) +  (m * 2 * (2 * 2)) +  ... +  (m * n * (n * n))

In [98]:
requirements

(|(x_{1, 1}, x_{1, 2}, ..., x_{1, n}, x_{2, 1}, x_{2, 2}, ..., x_{2, n}, ......, x_{m - 1, 1}, x_{m - 1, 2}, ..., x_{m - 1, n},(1 * 1), (2 * 2), ..., (n * n))| = |((1, 1), (1, 2), ..., (1, n), (2, 1), (2, 2), ..., (2, n), ......, (m, 1), (m, 2), ..., (m, n))|, |((1 * 1), (2 * 2), ..., (n * n))| = |((_b, 1), (_b, 2), ..., (_b, n))|, 1 in Integer, n in Integer, 1 in Integer, m in Integer, (1, 2, ..., (m - 1), m) = (1, 2, ..., m))

### Row-major vs column-major order

By our convention we the linearize a range of ranges of parameters according to row-major ordering.  This is what we used in the example above and is what `var_range` produced.  A column-major ordering is allowed, however, as long as it is employed consistently.  We repeat one of the above examples using column-major order.

In [99]:
x_11_to_mn__column_major = ExprRange(i, ExprRange(j, IndexedVar(x, (j, i)), one, n), one, m)

In [100]:
double_nested_lambda1_column_major = Lambda(x_11_to_mn__column_major, Mult(
    ExprRange(i, Add(ExprRange(j, Mult(i, j, IndexedVar(x, (j, i))), 
                               one, n)), 
              one, m)))

In [101]:
defaults.assumptions = [Equals(n, two), Equals(m, two)]

In [102]:
requirements = []
double_nested_lambda1_column_major.apply(*abcd.entries, requirements=requirements)

((1 * 1 * a) + (1 * 2 * b)) * ((2 * 1 * c) + (2 * 2 * d))

If the ordering is not consistent, such as using row-major order for the parameters but column-major order in the body of the lambda expression, an error will occur.

In [103]:
double_nested_lambda1_mixed_order = Lambda(x_11_to_mn, Mult(
    ExprRange(i, Add(ExprRange(j, Mult(i, j, IndexedVar(x, (j, i))), 
                               one, n)), 
              one, m)))

In [104]:
try:
    requirements = []
    double_nested_lambda1_mixed_order.apply(*abcd.entries, requirements=requirements)
    assert False, "Expected an LambdaApplicationError error."
except LambdaApplicationError as e:
    print("Expected error:", e)

Expected error: Failure to apply (x_{1, 1}, x_{1, 2}, ..., x_{1, n}, x_{2, 1}, x_{2, 2}, ..., x_{2, n}, ......, x_{m, 1}, x_{m, 2}, ..., x_{m, n}) -> (((1 * 1 * x_{1, 1}) +  (1 * 2 * x_{2, 1}) +  ... +  (1 * n * x_{n, 1})) *  ((2 * 1 * x_{1, 2}) +  (2 * 2 * x_{2, 2}) +  ... +  (2 * n * x_{n, 2})) *  ... *  ((m * 1 * x_{1, m}) +  (m * 2 * x_{2, m}) +  ... +  (m * n * x_{n, m}))) to (a, b, c, d) assuming {n = 2, m = 2}:
Improper replacement: Improper replacement of ((1 * 1 * x_{1, 1}) +  (1 * 2 * x_{2, 1}) +  ... +  (1 * n * x_{n, 1})), ((2 * 1 * x_{1, 2}) +  (2 * 2 * x_{2, 2}) +  ... +  (2 * n * x_{n, 2})), ..., ((m * 1 * x_{1, m}) +  (m * 2 * x_{2, m}) +  ... +  (m * n * x_{n, m})) via {(x_{1, 1}, x_{1, 2}, ..., x_{1, n}, x_{2, 1}, x_{2, 2}, ..., x_{2, n}, ......, x_{m, 1}, x_{m, 2}, ..., x_{m, n}): (a, b, c, d), x: {(x_{1, 1}, x_{1, 2}, ..., x_{1, n}, x_{2, 1}, x_{2, 2}, ..., x_{2, n}, ......, x_{m, 1}, x_{m, 2}, ..., x_{m, n})}}:
Failure to expand ((1 * 1 * x_{1, 1}) +  (1 * 2 * x_{2

## Singular and empty range reductions

When the length of an **ExprRange** is known to be empty or singular, in the process of performing a lambda application, the **ExprRange** will be eliminated appropriately within the **ExprTuple** in which it is contained.  

UPDATE: Obvious singular `ExprRange`s (where the start and end indices are the same expression) will be reduced at the time of `ExprRange` expansion.  Otherwise, it requires a separate simplification step (which can happen via auto-simplification during an instantiation).
For example,

In [105]:
tuple_with_range_lambda = Lambda(n, Add(a, ExprRange(i, Mult(i, i), one, n), c))

Applying this lambda to set $n : 0$ will eliminate this **ExprRange**, leaving just the $a$ and $c$ terms in this sum.

In [106]:
requirements = []
tuple_with_empty_range = tuple_with_range_lambda.apply(
    zero, requirements=requirements)

In [107]:
requirements

()

In [108]:
tuple_with_empty_range.simplification()

|- (a+ (1 * 1) +  (2 * 2) +  ... +  (0 * 0) + c) = (a + c)

This reduction works as long as it is known that the `end_index + 1 = start index`.

In [109]:
tuple_with_ij_range_lambda = Lambda(j, Add(a, ExprRange(b, Mult(b, b), i, j), c))

In [110]:
Add(subtract(i, one), one).simplification(assumptions=[InSet(i, Natural)])

{i in Natural} |- ((i - 1) + 1) = i

In [111]:
requirements = []
tuple_with_ij_empty_range = tuple_with_ij_range_lambda.apply(subtract(i, one), assumptions=[InSet(i, Natural)],
                                 requirements=requirements)

In [112]:
requirements

()

In [113]:
tuple_with_ij_empty_range.simplification(assumptions=[InSet(i, Natural)])

{i in Natural} |- (a+ (i * i) +  ((i + 1) * (i + 1)) +  ... +  ((i - 1) * (i - 1)) + c) = (a + c)

To reduce the **ExprRange** to a singular element, the `start_index` and `end_index` must match exactly.

In [114]:
requirements = []
tuple_with_ij_range_lambda.apply(i, requirements=requirements)

a + (i * i) + c

In [115]:
requirements

()

There are no other special **ExprRange** reductions, even if the expansion is obvious.  For example,

In [116]:
from proveit.numbers import two
tuple_with_range_lambda.apply(two)

a+ (1 * 1) +  ... +  (2 * 2) + c

Depending upon the circumstance, making the explicit expansion may not be desirable.  Such replacements should be made explicitly via the **theory system** rather than complicating the **core derivation system**.

In [117]:
%end exprrange_reductions

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>